In [ ]:
%matplotlib inline
import numpy as np
import scipy.stats as sps
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display
dtype = np.float64
from printmd import  *
float_formatter = "{:.4f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})
k = 15
sample_size = 20

## Типовой расчет по предмету МСА. Часть 1
### Остромецкий Дмитрий Вариант 15
**Выборка 1**

In [ ]:
def Y_l(i, j):
    i+=1
    j+=1
    return  i / (i + j)

def Y_b(j):
    j+=1
    return j * (k + 1) / k

Y_linear = np.fromfunction(Y_l, (4, 4), dtype=dtype)
Y_bias = np.fromfunction(Y_b, (4,), dtype=dtype)
norm = sps.norm.rvs(size=(sample_size, 4))
Y = np.matmul(norm, Y_linear.T) + Y_bias
pd.DataFrame(Y, columns=['y(1)', 'y(2)', 'y(4)', 'y(4)'])

**Выборка 2**

In [ ]:
def X_l(i, j):
    i+=1
    j+=1
    return  (i + 1)  / (i + j + 1)

def X_b(j):
    j+=1
    return j * (k + 3) / k

X_linear = np.fromfunction(X_l, (4, 4), dtype=dtype)
X_bias = np.fromfunction(X_b, (4,), dtype=dtype)
X = np.matmul(sps.norm.rvs(size=(sample_size, 4)), X_linear.T) + X_bias
pd.DataFrame(Y, columns=['x(1)', 'x(2)', 'x(4)', 'x(4)'])

1. **Найти векторы из оценок средних для обеих выборок**

In [ ]:
Y_m = Y.mean(axis=0)
X_m = X.mean(axis=0)

display(mdVector("\\hat{\\mu}_X", X_m))
display(mdVector("\\hat{\\mu}_Y", Y_m))

2. **Для элементов первой выборки найти первую и вторую главные компоненты**

In [ ]:
Y_D = np.cov(Y.T)
display(mdMatrix( "\\hat{D}_Y", Y_D))

In [ ]:

l, e = np.linalg.eig(Y_D)
plt.plot(range(len(l)), l / l.sum(), 'o-', linewidth=2, color='blue')
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.show()

In [ ]:
pc1 = np.matmul(Y - Y_m, e[0].T)
# plt.hist(np.matmul(Y - Y_m, e[0]), bins=5, density=True)
# plt.show()
print(pc1)

In [ ]:
pc3 = np.matmul(Y - Y_m, e[2].T)
plt.scatter(pc1, pc3)
plt.xlabel("Principal component 1")
plt.ylabel("Principal component 3")
plt.show()

3. **Для второй выборки найти линейную комбинацию центрированных 2-ой, 3-ей, 4-ой компонент x(2), x(3), x(4), имеющую наибольший коэффициент корреляции с центрированной первой компонентой x(1).**

In [ ]:
X_center = X - X_m
X_D = np.cov(X_center.T)

In [ ]:
from sklearn.cross_decomposition import CCA

cca = CCA(n_components=1)
cca.fit(X_center[ :,:1], X_center[ :,1:])
X_c, Y_c = cca.transform(X_center[ :,:1], X_center[ :,1:])
display(mdVector("\\vec{\\lambda}", cca.x_rotations_.ravel()))
display(mdVector("\\vec{\\beta}", cca.y_rotations_.ravel()))
plt.scatter(X_c, Y_c)
plt.xlabel(r"$\vec{v}$")
plt.ylabel(r"$\vec{u}$")
plt.show()

In [ ]:
display(mdMatrix( "\\hat{D}_X", X_D))

In [ ]:
alfa = cca.x_rotations_.ravel().reshape((1,1))
beta = cca.y_rotations_.ravel().reshape((3,1))
U_variance = alfa @ X_D[:1, :1] @ alfa
V_variance = beta.T @ X_D[1:, 1:] @ beta
cov = alfa @ X_D[:1, 1:] @ beta
p = cov / ((U_variance * V_variance) ** (1/2))
p[0][0]

4. **Объединить обе выборки в одну выборку, вставив вторую выборку под первой выборкой (x(1) под y(1) и т.д.). Получить выборку объема 40.**

In [ ]:
G = np.vstack((Y, X))
Z = np.ones((sample_size * 2, 2))
Z[sample_size:, 1] = 2

pd.DataFrame(G, columns=['g(1)', 'g(2)', 'g(4)', 'g(4)'])

5. **Оценить параметры линейной регрессии $E\vec{\gamma} = B\vec{z} = B\begin{pmatrix} 1 \\ z \end{pmatrix}$, где $B$ – матрица нагрузок порядка (4х2).**

In [ ]:
Q_gz = np.einsum("ki,kj->ij", G, Z)
Q_gg = np.einsum("ki,kj->ij", G, G)
Q_zz = np.einsum("ki,kj->ij", Z, Z)

B = Q_gz @ np.linalg.inv(Q_zz)
display(mdMatrix("\\hat{B}", B))
D = (Q_gg - B @ Q_zz @ B.T) / (sample_size * 2)
display(mdMatrix("\\hat{D}", D))

In [ ]:
plt.scatter(G[:sample_size,0], G[:sample_size, 1], c='#ff7f0e')
plt.scatter(G[sample_size:,0], G[sample_size:, 1], c='#2ca02c')
plt.show()


## 2 часть


In [ ]:
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
from numpy.linalg import inv

LABEL_COLOR_MAP = {0 : 'tab:blue',
                   1 : 'tab:red',
                   2 : 'tab:green',
                   3 : 'tab:cyan',
                   4 : 'tab:pink'
                   }


def create_sample(i):
    A = np.array([
                    [ 3 + 1/k, 0,       0      ],
                    [1,        2 + 1/k, 0      ],
                    [-1,       1,       1 + 1/k]
                ])
    norm = sps.norm.rvs(size=(sample_size, 3))

    return ((A @ norm.T).T + np.full(3, i/k))

def draw_clusters_3d(features, target):
    fig = plt.figure(figsize=(9, 7))
    ax=fig.add_subplot(111,projection='3d')
    ax.scatter(features[:, 0], features[:, 1], features[:, 2], c=[LABEL_COLOR_MAP[l] for l in target], marker='o')
    ax.set_xlabel(r"$\xi_1$")
    ax.set_ylabel(r"$\xi_2$")
    ax.set_zlabel(r"$\xi_3$")
    plt.show()


**Задача 1**

In [ ]:
n_1 = create_sample(1)
n_2 = create_sample(2)
n_3 = create_sample(3)

n_all = np.vstack((n_1, n_2, n_3))
n_13 = np.vstack((n_1, n_3))
target = np.full(sample_size * 3, 0)
target[sample_size:sample_size*2] = 1
target[sample_size*2:] = 2
draw_clusters_3d(n_all, target)

**Задача 2**

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=0, n_init="auto").fit(n_all)
draw_clusters_3d(n_all, kmeans.labels_)

**Задача 3**

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=0, n_init="auto").fit(n_all)
draw_clusters_3d(n_all, kmeans.labels_)

**Задача 4**

In [ ]:
agg_clustering = AgglomerativeClustering(distance_threshold=0, n_clusters=None)
agg_clustering.fit(n_all)

def plot_dendrogram(model, **kwargs):
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_, counts]).astype(float)

    dendrogram(linkage_matrix, **kwargs)

plt.figure(figsize=(12, 8))
plt.title("Дендрограмма")
plot_dendrogram(agg_clustering, truncate_mode=None, p=3)
plt.xlabel("Номер образца или кластера")
plt.ylabel("Расстояние")
plt.show()


**Задача 5**

In [ ]:
D_inv = inv(np.cov(n_13.T))
n_1_m = n_1.mean(axis=0)
n_2_m = n_3.mean(axis=0)

def classificate(xi):
    xi.T @ D_inv @ (n_1_m - n_2_m) >= (n_1_m + n_2_m) @ D_inv @ (n_1_m - n_2_m)